# 熵权法（Entropy Weight Method, EWM）
熵权法（也称信息熵权法）是一种基于**信息熵**思想的**客观赋权**方法：指标在方案之间差异越大，说明其提供的信息量越多，权重应越高；反之差异越小，权重越低。

## 方法核心
对于不同评价准则，该如何分配权重呢？一个合理的想法是：如果某一准则下各方案得分相近，可能说明该指标不能很好地区分待选方案，或者说该准则不能为我们的决策提供较多的信息，因而可以分配较低权重。

衡量某准则的得分是否相近，也就是衡量该准则是否能提供足够多的信息。从信息论/物理学的视角看：一致性更强（更“有序”）的信息对应更低的不确定性；反之更“混乱”的分布不确定性更高。我们借用**熵（Entropy）**刻画这种“混乱程度”，从而为不同准则自动分配权重向量。


## 数学图景

### 记号与基本假设（常见约定）
- 共有 $m$ 个方案（alternatives），$n$ 个准则（criteria）。
- 原始决策矩阵：

$$
X=(x_{ij})_{m\times n},\quad i=1,\dots,m;\ j=1,\dots,n
$$

其中 $x_{ij}$ 表示**方案 $A_i$** 在**准则 $C_j$** 下的观测值。

> 为了聚焦熵权法本身，本文档**假设所有准则均为效益型（最大值型）**。
>
> 成本型/中值型/区间型指标的正向化与标准化方式可参考 *TOPSIS.ipynb*（此处不展开）。


### Step 1：示例原始决策矩阵

为了便于理解，我们选取一个**刻意设计**的示例，使得“无区分度指标”和“强区分度指标”在结果中表现明显：
- $m=3$：方案 $A_1,A_2,A_3$
- $n=5$：准则 $C_1,C_2,C_3,C_4,C_5$

$$
X=
\begin{bmatrix}
80 & 100 & 50 & 60 & 10\\
82 & 100 & 50 & 85 & 40\\
78 & 100 & 50 & 95 & 90
\end{bmatrix}
$$

直观上：$C_2$、$C_3$ 列完全相同（几乎没有区分能力）；$C_5$ 差异很大（区分能力强）。


### Step 2：归一化（效益型：min–max）

由于各指标量纲与尺度可能不同，需要先无量纲化。对效益型指标，常用 min–max 归一化：

$$
r_{ij} = \frac{x_{ij}-\min\limits_{1\le i\le m}x_{ij}}{\max\limits_{1\le i\le m}x_{ij}-\min\limits_{1\le i\le m}x_{ij}},\quad 0\le r_{ij}\le 1
$$

得到归一化矩阵 $R=(r_{ij})_{m\times n}$。

> 注：若某列满足 $\max_i x_{ij}=\min_i x_{ij}$，说明该指标对所有方案取值完全一致，分母为 0。
>
> 这正对应“无区分度指标”，实践中可做特判（如直接令该指标权重为 0 或加入极小量 $\varepsilon$）。本文档在理论层面强调其含义。


### Step 3：构造比例矩阵 $P=(p_{ij})$

熵的计算需要把每个指标列视作一个“分布”。因此对每个准则 $C_j$，把 $R$ 的第 $j$ 列转为比例：

$$
p_{ij} = \frac{r_{ij}}{\sum\limits_{i=1}^{m} r_{ij}},\quad i=1,\dots,m;\ j=1,\dots,n
$$

得到比例矩阵 $P=(p_{ij})_{m\times n}$。对任意固定的 $j$，都有：

$$
\sum_{i=1}^{m} p_{ij}=1,\quad p_{ij}\ge 0.
$$

直观理解：$p_{ij}$ 描述在准则 $C_j$ 下，方案 $A_i$ 的相对“占比”。


### Step 4：计算信息熵 $e_j$

信息熵刻画分布的均匀程度：分布越均匀，熵越大；越不均匀，熵越小。

为了使 $e_j\in[0,1]$，引入常数：

$$
k=\frac{1}{\ln m}.
$$

第 $j$ 个准则的信息熵定义为：

$$
e_j=-k\sum_{i=1}^{m} p_{ij}\ln(p_{ij}),\quad j=1,\dots,n.
$$

并有：

$$
0\le e_j\le 1.
$$

**解释：**
- 若某指标在各方案间差异很小，则 $p_{ij}$ 近似均匀，$e_j$ 接近 1（信息量少）。
- 若某指标差异显著，则 $p_{ij}$ 明显不均匀，$e_j$ 更小（信息量多）。


### Step 5：冗余度 $d_j$ 与权重 $w_j$

熵越大代表有效信息越少，因此定义冗余度（也称差异系数/信息效用）：

$$
d_j = 1-e_j.
$$

最后将 $d_j$ 归一化得到权重：

$$
w_j = \frac{d_j}{\sum\limits_{j=1}^{n} d_j}=\frac{1-e_j}{\sum\limits_{j=1}^{n}(1-e_j)},\quad j=1,\dots,n.
$$

性质：
$$
w_j\ge 0,\quad \sum_{j=1}^{n} w_j = 1.
$$

**结合示例的直观判断（无需计算也能“预判”）：**
- $C_2$、$C_3$ 对所有方案取值相同 → 分布最均匀 → 熵最大 $e_j\approx 1$ → 冗余度 $d_j\approx 0$ → 权重应接近 0。
- $C_5$ 差异最大 → 分布最不均匀 → 熵显著偏小 → 冗余度最大 → 权重应最大。


### 小结：熵权法的“因果链”

熵权法的计算链条可以概括为：

$$
X\longrightarrow R\longrightarrow P\longrightarrow e\longrightarrow d\longrightarrow w
$$

从原始数据出发，通过归一化与比例化得到“分布”，再用熵刻画其均匀程度，最终把“差异信息”转化为权重。


## 适用性分析

### 适用场景
- 多指标评价/排序需要权重，但不希望或无法引入主观判断；
- 指标数据来源较客观（统计、测量、量化指标体系等）；
- 希望权重能反映指标在样本中的**区分能力/信息量**。

### 优点
- **客观性强：**权重由数据驱动，减少人为偏差；
- **可解释：**权重与“离散程度/信息量”一一对应；
- **易实现：**公式结构清晰，计算流程明确。

### 局限性与注意事项
- **不等同于“重要性”：**熵权法衡量的是“区分度/信息量”，不代表决策者价值偏好下的业务重要性；
- **对数据分布敏感：**样本太少或存在异常值时，离散度可能被极端值放大，导致权重偏移；
- **完全一致指标的处理：**若 $\max=\min$，需在实现中做特判或加入 $\varepsilon$；
- **指标类型问题：**现实常含成本型/区间型/中值型指标，应先正向化再进入熵权流程（见 *TOPSIS.ipynb*）。
